In [ ]:
import requests
import time
import logging
import pandas as pd
from requests.exceptions import RequestException
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML
import io
import zipfile
import base64

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Most common SEC filing document types
COMMON_DOC_TYPES = [
    "10-K", "10-Q", "8-K", "S-1", "S-4", "13F-HR", "4", "SC 13G", "13D",
    "DEF 14A", "6-K", "20-F", "40-F", "F-1", "F-4", "144", "424B2", "FWP"
]

def get_cik(ticker):
    url = "https://www.sec.gov/files/company_tickers.json"
    headers = {
        "User-Agent": "YourName yourname@example.com"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        data = response.json()
 o       for item in data.values():
            if item['ticker'] == ticker.upper():
                return str(item['cik_str']).zfill(10)
        logging.warning(f"CIK not found for ticker {ticker}")
        return None
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while retrieving CIK: {e}")
    except ValueError as e:
        logging.error(f"JSON decoding error while retrieving CIK: {e}")
    except Exception as e:
        logging.error(f"Unexpected error while retrieving CIK: {e}")
    return None

def get_recent_filings(ticker, doc_types, start_year, end_year):
    cik = get_cik(ticker)
    if not cik:
        logging.error(f"Unable to retrieve CIK for ticker {ticker}")
        return None

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    headers = {
        "User-Agent": "YourName yourname@example.com"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        data = response.json()
        filings = data.get('filings', {}).get('recent', {})

        if not filings:
            logging.warning("No recent filings found.")
            return None

        # Filter by document type and year range
        filtered_filings = []
        for i in range(len(filings['accessionNumber'])):
            filing_year = int(filings['filingDate'][i][:4])
            if (not doc_types or filings['form'][i] in doc_types) and \
               (start_year <= filing_year <= end_year):
                filtered_filings.append({
                    'accession_number': filings['accessionNumber'][i],
                    'filing_date': filings['filingDate'][i],
                    'form': filings['form'][i],
                    'primary_document': filings['primaryDocument'][i]
                })

        if not filtered_filings:
            logging.info("No filings match the specified criteria.")
            return None

        return filtered_filings
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while retrieving filings: {e}")
    except ValueError as e:
        logging.error(f"JSON decoding error while retrieving filings: {e}")
    except Exception as e:
        logging.error(f"Unexpected error while retrieving filings: {e}")
    return None

def download_filing_document(cik, accession_number, primary_document):
    base_url = "https://www.sec.gov/Archives/edgar/data"
    accession_number_no_dashes = accession_number.replace("-", "")
    url = f"{base_url}/{cik}/{accession_number_no_dashes}/{primary_document}"
    headers = {
        "User-Agent": "YourName yourname@example.com"
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.content
    except requests.exceptions.RequestException as e:
        logging.error(f"Network error while downloading document: {e}")
    except Exception as e:
        logging.error(f"Unexpected error while downloading document: {e}")
    return None

def create_download_link(content, filename):
    b64 = base64.b64encode(content).decode()
    return f'<a href="data:application/octet-stream;base64,{b64}" download="{filename}">Download {filename}</a>'

def create_zip_download_link(files):
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for filename, content in files:
            zip_file.writestr(filename, content)

    b64 = base64.b64encode(zip_buffer.getvalue()).decode()
    return f'<a href="data:application/zip;base64,{b64}" download="sec_filings.zip">Download SEC Filings (ZIP)</a>'

def create_ui():
    ticker_input = widgets.Text(description="Company Ticker:")
    doc_type_select = widgets.SelectMultiple(
        options=COMMON_DOC_TYPES,
        description="Document Types:",
        rows=5
    )
    start_year = widgets.IntText(description="Start Year:", value=2018)
    end_year = widgets.IntText(description="End Year:", value=2023)
    search_button = widgets.Button(description="Search Filings")
    output = widgets.Output()

    filings_select = widgets.SelectMultiple(description="Select Filings:", options=[])
    download_button = widgets.Button(description="Download Selected Filings")
    status_output = widgets.Output()

    def on_search_click(b):
        with output:
            clear_output()
            filings = get_recent_filings(ticker_input.value, list(doc_type_select.value), start_year.value, end_year.value)
            if filings:
                options = [(f"{f['filing_date']} - {f['form']}", f) for f in filings]
                filings_select.options = options
                print(f"Found {len(filings)} filings.")
            else:
                print("No filings found or an error occurred.")

    def on_download_click(b):
        with status_output:
            clear_output()
            if filings_select.value:
                cik = get_cik(ticker_input.value)
                files_to_download = []
                for filing in filings_select.value:
                    content = download_filing_document(cik, filing['accession_number'], filing['primary_document'])
                    if content:
                        filename = f"{filing['accession_number']}_{filing['primary_document']}"
                        files_to_download.append((filename, content))
                    else:
                        print(f"Failed to download filing: {filing['accession_number']}")

                if files_to_download:
                    if len(files_to_download) == 1:
                        filename, content = files_to_download[0]
                        download_link = create_download_link(content, filename)
                    else:
                        download_link = create_zip_download_link(files_to_download)

                    display(HTML(download_link))
                else:
                    print("No files were successfully downloaded.")
            else:
                print("Please select at least one filing to download.")

    search_button.on_click(on_search_click)
    download_button.on_click(on_download_click)

    ui = widgets.VBox([
        widgets.HBox([ticker_input, doc_type_select]),
        widgets.HBox([start_year, end_year, search_button]),
        output,
        filings_select,
        download_button,
        status_output
    ])

    return ui

# Create and display the UI
ui = create_ui()
display(ui)

print("UI with mobile-friendly download options is now ready.")

UI with mobile-friendly download options is now ready.
